In [1]:
import pandas as pd
import numpy as np

import xml.etree.ElementTree as ET
import networkx as nx

In [2]:
tree = ET.parse('padgett.xml')

In [3]:
root = tree.getroot()

In [4]:
for child in root.findall('MetaNetwork'):
    print(child)

<Element 'MetaNetwork' at 0x00000226AFCCF860>


In [5]:
child_1 = root[0] # Спускаемся в MetaNetwork

In [6]:
for child in child_1:
    print(child)

<Element 'nodes' at 0x00000226AFCCF8B0>
<Element 'networks' at 0x00000226AFCD33B0>


In [7]:
child_2 = child_1[1] # Спускаемся в networks

In [8]:
for child in child_2: # Спускаемся в PADGM
    if (child.attrib['id'] == 'PADGM'):
        child_3 = child

In [9]:
df = pd.DataFrame()
for child in child_3:
    df = df.append(child.attrib, ignore_index = True)
    
df.head()

,source,target,type,value
0,ACCIAIUOL,ACCIAIUOL,double,0.0000
1,ACCIAIUOL,ALBIZZI,double,0.0000
2,ACCIAIUOL,BARBADORI,double,0.0000
3,ACCIAIUOL,BISCHERI,double,0.0000
4,ACCIAIUOL,CASTELLAN,double,0.0000


In [10]:
df['value'] = df['value'].astype(float)

In [11]:
df.drop(columns = 'type', inplace = True)

In [12]:
def pagerank(G, alpha=0.85, personalization=None,
			max_iter=100, tol=1.0e-6, nstart=None, weight='weight',
			dangling=None):
	"""Return the PageRank of the nodes in the graph.

	PageRank computes a ranking of the nodes in the graph G based on
	the structure of the incoming links. It was originally designed as
	an algorithm to rank web pages.

	Parameters
	----------
	G : graph
	A NetworkX graph. Undirected graphs will be converted to a directed
	graph with two directed edges for each undirected edge.

	alpha : float, optional
	Damping parameter for PageRank, default=0.85.

	personalization: dict, optional
	The "personalization vector" consisting of a dictionary with a
	key for every graph node and nonzero personalization value for each node.
	By default, a uniform distribution is used.

	max_iter : integer, optional
	Maximum number of iterations in power method eigenvalue solver.

	tol : float, optional
	Error tolerance used to check convergence in power method solver.

	nstart : dictionary, optional
	Starting value of PageRank iteration for each node.

	weight : key, optional
	Edge data key to use as weight. If None weights are set to 1.

	dangling: dict, optional
	The outedges to be assigned to any "dangling" nodes, i.e., nodes without
	any outedges. The dict key is the node the outedge points to and the dict
	value is the weight of that outedge. By default, dangling nodes are given
	outedges according to the personalization vector (uniform if not
	specified). This must be selected to result in an irreducible transition
	matrix (see notes under google_matrix). It may be common to have the
	dangling dict to be the same as the personalization dict.

	Returns
	-------
	pagerank : dictionary
	Dictionary of nodes with PageRank as value

	Notes
	-----
	The eigenvector calculation is done by the power iteration method
	and has no guarantee of convergence. The iteration will stop
	after max_iter iterations or an error tolerance of
	number_of_nodes(G)*tol has been reached.

	The PageRank algorithm was designed for directed graphs but this
	algorithm does not check if the input graph is directed and will
	execute on undirected graphs by converting each edge in the
	directed graph to two edges.

	
	"""
	if len(G) == 0:
		return {}

	if not G.is_directed():
		D = G.to_directed()
	else:
		D = G

	# Create a copy in (right) stochastic form
	W = nx.stochastic_graph(D, weight=weight)
	N = W.number_of_nodes()

	# Choose fixed starting vector if not given
	if nstart is None:
		x = dict.fromkeys(W, 1.0 / N)
	else:
		# Normalized nstart vector
		s = float(sum(nstart.values()))
		x = dict((k, v / s) for k, v in nstart.items())

	if personalization is None:

		# Assign uniform personalization vector if not given
		p = dict.fromkeys(W, 1.0 / N)
	else:
		missing = set(G) - set(personalization)
		if missing:
			raise NetworkXError('Personalization dictionary '
								'must have a value for every node. '
								'Missing nodes %s' % missing)
		s = float(sum(personalization.values()))
		p = dict((k, v / s) for k, v in personalization.items())

	if dangling is None:

		# Use personalization vector if dangling vector not specified
		dangling_weights = p
	else:
		missing = set(G) - set(dangling)
		if missing:
			raise NetworkXError('Dangling node dictionary '
								'must have a value for every node. '
								'Missing nodes %s' % missing)
		s = float(sum(dangling.values()))
		dangling_weights = dict((k, v/s) for k, v in dangling.items())
	dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]

	# power iteration: make up to max_iter iterations
	for _ in range(max_iter):
		xlast = x
		x = dict.fromkeys(xlast.keys(), 0)
		danglesum = alpha * sum(xlast[n] for n in dangling_nodes)
		for n in x:

			# this matrix multiply looks odd because it is
			# doing a left multiply x^T=xlast^T*W
			for nbr in W[n]:
				x[nbr] += alpha * xlast[n] * W[n][nbr][weight]
			x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n]

		# check convergence, l1 norm
		err = sum([abs(x[n] - xlast[n]) for n in x])
		if err < N*tol:
			return x
	raise NetworkXError('pagerank: power iteration failed to converge '
						'in %d iterations.' % max_iter)


In [13]:
source_names = df['source'].unique()
target_names = df['target'].unique()

In [14]:
A = np.empty((len(source_names), len(target_names)))

for i in range (len(source_names)):
    df_i = df[df['source'] == source_names[i]]
    for j in range (len(target_names)):
        A[i][j] = df_i[df['target'] == target_names[j]]['value']

<ipython-input-14-bdf22f8fbd6c>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  A[i][j] = df_i[df['target'] == target_names[j]]['value']


In [15]:
G = nx.from_numpy_matrix(A)

In [16]:
result = []
for i in range(len(pagerank(G))):
    result.append(pagerank(G)[i])

In [17]:
pd.DataFrame({'family': source_names, 'rank': result}).sort_values(by = 'rank', ascending = False)

,family,rank
8,MEDICI,0.144372
6,GUADAGNI,0.097423
14,STROZZI,0.087226
1,ALBIZZI,0.078340
15,TORNABUON,0.070574
12,RIDOLFI,0.068886
4,CASTELLAN,0.068644
3,BISCHERI,0.068180
10,PERUZZI,0.067203
13,SALVIATI,0.060697
